## Import

In [1]:
import pandas as pd
import numpy as np
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, AdamW
from tqdm import tqdm

# CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Data Preprocessing

In [3]:
# 데이터 로드
data = pd.read_csv('train.csv')

# 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained('skt/kogpt2-base-v2', eos_token='</s>')

# 데이터 포맷팅 및 토크나이징
formatted_data = []
for _, row in tqdm(data.iterrows()):
    for q_col in ['질문_1', '질문_2']:
        for a_col in ['category', '답변_1', '답변_2', '답변_3', '답변_4', '답변_5']:
            # 질문과 답변 쌍을 </s> token으로 연결
            input_text = row[q_col] + tokenizer.eos_token + row[a_col]
            input_ids = tokenizer.encode(input_text, return_tensors='pt')
            formatted_data.append(input_ids)
print('Done.')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
644it [00:02, 257.96it/s]

Done.


## Model Fine-tuning

In [ ]:
# 모델 로드
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
model.to(device) # 모델을 GPU단으로 이동

# 모델 학습 하이퍼파라미터(Hyperparameter) 세팅
# 실제 필요에 따라 조정하세요.
CFG = {
    'LR' : 2e-5, # Learning Rate
    'EPOCHS' : 10, # 학습 Epoch
}

# 모델 학습 설정
optimizer = AdamW(model.parameters(), lr=CFG['LR'])
model.train()

# 모델 학습
for epoch in range(CFG['EPOCHS']):
    total_loss = 0
    progress_bar = tqdm(enumerate(formatted_data), total=len(formatted_data))
    for batch_idx, batch in progress_bar:
        # 데이터를 GPU단으로 이동
        batch = batch.to(device)
        outputs = model(batch, labels=batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

        # 진행률 표시줄에 평균 손실 업데이트
        progress_bar.set_description(f"Epoch {epoch+1} - Avg Loss: {total_loss / (batch_idx+1):.4f}")

    # 에폭의 평균 손실을 출력
    print(f"Epoch {epoch+1}/{CFG['EPOCHS']}, Average Loss: {total_loss / len(formatted_data)}")

# 모델 저장
model.save_pretrained("./hansoldeco-kogpt2")
tokenizer.save_pretrained("./hansoldeco-kogpt2")

C:\Users\82109\anaconda3\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1 - Avg Loss: 2.8083:  73%|██████████████████████████████████▉             | 4693/6440 [2:30:10<55:54,  1.92s/it]


## Model Inference

In [ ]:
# 저장된 Fine-tuned 모델과 토크나이저 불러오기
model_dir = "./hansoldeco-kogpt2"
model = GPT2LMHeadModel.from_pretrained(model_dir)
model.to(device)
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_dir)

# Inference를 위한 test.csv 파일 로드
test = pd.read_csv('./test.csv')

# test.csv의 '질문'에 대한 '답변'을 저장할 리스트
preds = []

# '질문' 컬럼의 각 질문에 대해 답변 생성
for test_question in tqdm(test['질문']):
    # 입력 텍스트를 토큰화하고 모델 입력 형태로 변환
    input_ids = tokenizer.encode(test_question + tokenizer.eos_token, return_tensors='pt')

    # 답변 생성
    output_sequences = model.generate(
        input_ids=input_ids.to(device),
        max_length=300,
        temperature=0.9,
        top_k=1,
        top_p=0.9,
        repetition_penalty=1.2,
        do_sample=True,
        num_return_sequences=1
    )

    # 생성된 텍스트(답변) 저장
    for generated_sequence in output_sequences:
        full_text = tokenizer.decode(generated_sequence, skip_special_tokens=False)
        # 질문과 답변의 사이를 나타내는 eos_token (</s>)를 찾아, 이후부터 출력
        answer_start = full_text.find(tokenizer.eos_token) + len(tokenizer.eos_token)
        answer_only = full_text[answer_start:].strip()
        answer_only = answer_only.replace('\n', ' ')
        preds.append(answer_only)

In [11]:
test_data = pd.read_csv('지원누나꺼에서답변하나만손봄.csv')

In [12]:
preds_3 = test_data["Answer"]
pred_embeddings = model.encode(preds_3)
pred_embeddings.shape

(130, 512)

## Submission

In [5]:
!pip install sentence_transformers

  Obtaining dependency information for sentence_transformers from https://files.pythonhosted.org/packages/06/97/57afa3d05801b6b9305f96a7ce5995e12c1d2ba25ce66747de107816b0b5/sentence_transformers-2.3.1-py3-none-any.whl.metadata
  Obtaining dependency information for sentencepiece from https://files.pythonhosted.org/packages/cc/07/d6951e3b4079df819d76353302fc3e76835252e7e0b6366f96a03d87ea5f/sentencepiece-0.1.99-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/132.8 kB ? eta -:--:--
   --- ------------------------------------ 10.2/132.8 kB ? eta -:--:--
   ------------------------------------- -- 122.9/132.8 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 132.8/132.8 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/977.5 kB ? eta -:--:--
   ---------------- ---------------------- 419.8/977.5 kB 13.2 MB/s eta 0:00:01
   --------------------------------- ----- 849.9/977.5 kB 10.8 MB/s eta 0:00:01
   ---------------

In [13]:
# Test 데이터셋의 모든 질의에 대한 답변으로부터 512 차원의 Embedding Vector 추출
# 평가를 위한 Embedding Vector 추출에 활용하는 모델은 'distiluse-base-multilingual-cased-v1' 이므로 반드시 확인해주세요.
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# 생성한 모든 응답(답변)으로부터 Embedding Vector 추출
preds_3 = test_data["Answer"]
pred_embeddings = model.encode(preds_3)
pred_embeddings.shape

(130, 512)

In [14]:
submit = pd.read_csv('./sample_submission.csv')
# 제출 양식 파일(sample_submission.csv)을 활용하여 Embedding Vector로 변환한 결과를 삽입
submit.iloc[:,1:] = pred_embeddings
submit.head()

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,0.015054,0.032182,0.045047,-0.033445,0.074882,-0.000633,-0.036463,0.055597,0.045907,...,-0.014158,0.015436,-0.016378,-0.023483,-0.006140,0.018503,0.016631,-0.005467,0.022610,0.053644
1,TEST_001,-0.025969,0.028366,0.021325,-0.006204,0.099643,-0.052744,0.016071,-0.068605,-0.051410,...,-0.019498,-0.047119,0.020995,-0.053973,-0.016779,0.025215,-0.023197,-0.040746,-0.013399,0.021817
2,TEST_002,-0.007217,0.019709,-0.022726,-0.009670,0.131308,-0.036227,0.007529,-0.039610,0.066471,...,-0.025251,0.000736,0.085899,-0.000033,-0.017271,0.020787,-0.004494,-0.013021,-0.059203,0.024421
3,TEST_003,-0.006465,0.034693,-0.026738,0.019800,0.052408,-0.051949,-0.080758,-0.015945,0.000110,...,-0.039307,-0.002394,0.026957,-0.043071,0.004968,0.053339,-0.046710,-0.029743,-0.025439,0.089134
4,TEST_004,0.010277,-0.007995,-0.003341,-0.004290,0.100152,-0.040631,0.053409,0.024780,-0.010998,...,-0.015132,0.008700,0.014739,0.009830,-0.014686,0.023440,-0.017766,0.047517,-0.018717,0.058538


In [ ]:
---

In [15]:
# 리더보드 제출을 위한 csv파일 생성
submit.to_csv('./submit.csv', index=False)